In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is sheenu and I am a AI intern")])

AIMessage(content="Hi Sheenu! It's nice to meet you. Welcome to the world of AI!  \n\nWhat kind of work are you doing as an AI intern? Are you working on any exciting projects?  \n\nI'm always eager to learn more about what other AI models are up to. 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 27, 'total_tokens': 93, 'completion_time': 0.136338715, 'prompt_time': 0.003104779, 'queue_time': None, 'total_time': 0.139443494}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-be784efc-8399-46b9-b059-2cf76f999946-0', usage_metadata={'input_tokens': 27, 'output_tokens': 66, 'total_tokens': 93})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is sheenu and I am a AI intern"),
        AIMessage(content="Hello Sheenu! It's nice to meet you.  \n\nWelcome to the world of AI"),
        HumanMessage(content="what's my name and profession?")
    ]
)

AIMessage(content="You told me your name is Sheenu and that you are an AI intern. 😊  \n\nIs there anything else you'd like to tell me about yourself?\n", response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 65, 'total_tokens': 101, 'completion_time': 0.072960171, 'prompt_time': 0.004506668, 'queue_time': None, 'total_time': 0.07746683900000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e21cc7d1-aef8-414b-8454-9573a43d18bb-0', usage_metadata={'input_tokens': 65, 'output_tokens': 36, 'total_tokens': 101})

### MESSAGE HISTORY
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [7]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history)  #now we can able to interact with out model based on message history

this function will ensure that every session will store different chat history

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is sheenu and I am a Chief AI Engineer")],
    config=config
)

In [10]:
response

AIMessage(content="Hello Sheenu, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you working on these days?  \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 28, 'total_tokens': 83, 'completion_time': 0.112817265, 'prompt_time': 0.002623169, 'queue_time': None, 'total_time': 0.115440434}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-96cd8dc4-e5ce-47d1-b606-bd517c755b5a-0', usage_metadata={'input_tokens': 28, 'output_tokens': 55, 'total_tokens': 83})

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Sheenu.  \n\nI remember that from our earlier conversation! 😊  How can I help you today? \n', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 97, 'total_tokens': 126, 'completion_time': 0.058710977, 'prompt_time': 0.006403987, 'queue_time': None, 'total_time': 0.065114964}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed8cdc4e-5ef5-4fd9-a844-0762cff2b870-0', usage_metadata={'input_tokens': 97, 'output_tokens': 29, 'total_tokens': 126})

### prompt template
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [12]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [13]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is sheenu")]})

AIMessage(content="Hello Sheenu! 👋\n\nIt's nice to meet you. What can I do for you today? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 37, 'total_tokens': 64, 'completion_time': 0.054480048, 'prompt_time': 0.002909889, 'queue_time': None, 'total_time': 0.057389937}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c69b29a8-c54f-4c1e-ab77-836ac86e7232-0', usage_metadata={'input_tokens': 37, 'output_tokens': 27, 'total_tokens': 64})

In [14]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [15]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is sheenu")],
    config=config
)

response

AIMessage(content="Hello Sheenu, it's nice to meet you! 👋\n\nIs there anything I can help you with today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 37, 'total_tokens': 66, 'completion_time': 0.058707586, 'prompt_time': 0.002897699, 'queue_time': None, 'total_time': 0.061605284999999996}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b73f3410-d676-4c28-a11f-415fd5f21ede-0', usage_metadata={'input_tokens': 37, 'output_tokens': 29, 'total_tokens': 66})

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sheenu!  😄  \n\nI remember that you told me.  Is there anything else I can help you with? \n'

In [17]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is sheenu")],"language":"hindi"})
response.content

'नमस्ते, शीनु! 😊 \n\nआप कैसे हैं? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

input_messages_key="messages" parameter specifies the key in the input dictionary where the messages are stored. This key tells the function where to find the list of messages within the input data structure.

In [20]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! 😊 \n\nआप कैसे हैं?  \n'

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [22]:
response.content

'आपका नाम कृष्ण है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [25]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=95,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content="hi! I'm bob"),
 AIMessage(content='hi!'),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You like vanilla ice cream!  🍦😊  \n\n\n\nIs there anything else I can help you with?\n'

In [27]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my favourite icecream?")],
        "language": "English",
    },
    config=config,
)

response.content

'You said you liked vanilla ice cream!  🍦  \n\nIs that your favorite? 😊\n'